In [10]:
import numpy as np
import torch

In [11]:

def conv(input, kernel, padding=0, stride=1):
    input_height, input_width = input.shape
    kernel_height, kernel_width = kernel.shape
    
    # Calculate output dimensions
    output_height = (input_height - kernel_height + 2 * padding) // stride + 1
    output_width = (input_width - kernel_width + 2 * padding) // stride + 1
    
    # Apply padding if necessary
    if padding > 0:
        padded_input = np.zeros((input_height + 2 * padding, input_width + 2 * padding))
        padded_input[padding:padding + input_height, padding:padding + input_width] = input
    else:
        padded_input = input
    
    # Initialize output matrix
    output = np.zeros((output_height, output_width))
    
    # Perform convolution
    for y in range(0, output_height):
        for x in range(0, output_width):
            region = padded_input[y * stride:y * stride + kernel_height, x * stride:x * stride + kernel_width]
            output[y, x] = np.sum(region * kernel)
    
    return output


In [12]:

input_matrix = np.array([
    [1, 2, 3, 0, 1],
    [4, 5, 6, 1, 0],
    [7, 8, 9, 2, 3],
    [1, 0, 1, 4, 5],
    [2, 3, 0, 6, 7]
])


kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

output = conv(input_matrix, kernel, padding=1, stride=1)
print(output)


[[ -7.  -4.   6.   8.   1.]
 [-15.  -6.  12.  14.   3.]
 [-13.  -4.   6.   8.   7.]
 [-11.   0.  -1.  -5.  12.]
 [ -3.   2.  -7. -11.  10.]]


In [13]:
def maxPool(input, pool_size=2, stride=1):
    output_height = (input.shape[0] - pool_size) // stride + 1
    output_width = (input.shape[1] - pool_size) // stride + 1
    output = np.zeros((output_height, output_width))
    
    for x in range(0, output_height):
        for y in range(0, output_width):
            region = input[x*stride:x*stride+pool_size, y*stride:y*stride+pool_size]
            output[x, y] = np.max(region)
    
    return output

In [14]:
print(maxPool(output))

[[-4. 12. 14. 14.]
 [-4. 12. 14. 14.]
 [ 0.  6.  8. 12.]
 [ 2.  2. -1. 12.]]


In [15]:
import numpy as np

class ConvLayer:
    def __init__(self, input, kernel, padding=0, stride=1):
        self.input = input
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
    
    def conv(self):
        input_height, input_width = self.input.shape
        kernel_height, kernel_width = self.kernel.shape
        
        output_height = (input_height - kernel_height + 2 * self.padding) // self.stride + 1
        output_width = (input_width - kernel_width + 2 * self.padding) // self.stride + 1
        
        if self.padding > 0:
            padded_input = np.zeros((input_height + 2 * self.padding, input_width + 2 * self.padding))
            padded_input[self.padding:self.padding + input_height, self.padding:self.padding + input_width] = self.input
        else:
            padded_input = self.input
        
        output = np.zeros((output_height, output_width))
        
        for y in range(0, output_height):
            for x in range(0, output_width):
                region = padded_input[y * self.stride:y * self.stride + kernel_height, x * self.stride:x * self.stride + kernel_width]
                output[y, x] = np.sum(region * self.kernel)
        
        return output
    
    def maxPool(self, pool_size=2, stride=1):
        input_height, input_width = self.input.shape
        
        output_height = (input_height - pool_size) // stride + 1
        output_width = (input_width - pool_size) // stride + 1
        output = np.zeros((output_height, output_width))
        
        for x in range(0, output_height):
            for y in range(0, output_width):
                region = self.input[x*stride:x*stride+pool_size, y*stride:y*stride+pool_size]
                output[x, y] = np.max(region)
        
        return output




In [16]:
input_matrix = np.array([
    [1, 2, 3, 0, 1],
    [4, 5, 6, 1, 0],
    [7, 8, 9, 2, 3],
    [1, 0, 1, 4, 5],
    [2, 3, 0, 6, 7]
])

kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

conv_layer = ConvLayer(input_matrix, kernel, padding=1, stride=1)
conv_output = conv_layer.conv()
max_pool_output = conv_layer.maxPool(pool_size=2, stride=2)

print("Convolution Output:\n", conv_output)
print("\nMax Pooling Output:\n", max_pool_output)

Convolution Output:
 [[ -7.  -4.   6.   8.   1.]
 [-15.  -6.  12.  14.   3.]
 [-13.  -4.   6.   8.   7.]
 [-11.   0.  -1.  -5.  12.]
 [ -3.   2.  -7. -11.  10.]]

Max Pooling Output:
 [[5. 6.]
 [8. 9.]]


In [18]:
import torch
import torch.nn as nn
input_tensor = torch.tensor(input_matrix, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

conv_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=0)
with torch.no_grad():
    conv_layer.weight = torch.nn.Parameter(torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0))
output_tensor = conv_layer(input_tensor)

print("Output Tensor:")
print(output_tensor)
print("Output Tensor Shape:", output_tensor.shape)


Output Tensor:
tensor([[[[-6.1478, 11.8522, 13.8522],
          [-4.1478,  5.8522,  7.8522],
          [-0.1478, -1.1478, -5.1478]]]], grad_fn=<ConvolutionBackward0>)
Output Tensor Shape: torch.Size([1, 1, 3, 3])
